In [4]:
import os
import cv2
from ultralytics import YOLO
import csv
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
conf = 0.7
dir = "datanew1.v2i.multiclass"
dir2 = "test"
# Path to the directory containing images
input_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}'  # Directory containing images
output_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}_detect_{conf}'  # Output directory for annotated images
output_file = f"C:/Users/User/Desktop/dataset/{dir}/{dir2}_keypoints_{conf}.csv"  # Output CSV file for keypoints data

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model

def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for j in range(17):  # Assuming there are 17 keypoints
            x, y = r.keypoints.xyn[i][j].numpy()
            combined_results.append((x, y))
    box = results[0].boxes.xywhn.numpy().reshape(-1, 2)
    list_of_tuples = [tuple(row) for row in box]
    combined_results.extend(list_of_tuples)
    return combined_results

# Process each PNG image in the input directory
all_keypoints_data = []
# Get all files in input_dir
all_files = os.listdir(input_dir)
# Filter out only files that end with .png and are valid files
image_files = [filename for filename in all_files if os.path.isfile(os.path.join(input_dir, filename)) and filename.lower().endswith('.jpg')]
# Sort the filtered image files
image_files.sort()

for idx, filename in enumerate(image_files):
    input_path = os.path.join(input_dir, filename)
    
    # Output path remains the same as input path, with '_detect_confidence' appended
    output_path = os.path.join(output_dir, filename)

    # Load the image
    image = cv2.imread(input_path)

    # Perform pose detection
    results = model(image, conf=conf)

    if len(results[0].boxes) == 1:
        # Draw the results on the image
        annotated_image = results[0].plot()
        # Save the annotated image with the original filename
        cv2.imwrite(output_path, annotated_image)
        print(f"Processed and saved: {output_path}")
        keypoints_data = process_image(results)
        keypoints_data.insert(0, filename)  # Use the original filename
        all_keypoints_data.append(keypoints_data)
    else:
        keypoints_data = [filename] + [None] * 19  # Assuming 17 keypoints with x,y pairs
        all_keypoints_data.append(keypoints_data)

# Write keypoints data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}" for i in range(17)] + ["xb, yb"] + ["w, h"]  # Assuming 17 keypoints with x,y pairs
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)



0: 640x640 (no detections), 202.4ms
Speed: 6.2ms preprocess, 202.4ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 141.7ms
Speed: 2.0ms preprocess, 141.7ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/test_detect_0.7\1006_jpg.rf.60a37a126e188d5d98c191ccdfeb3d38.jpg

0: 640x640 1 person, 136.4ms
Speed: 2.0ms preprocess, 136.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/test_detect_0.7\100_jpg.rf.bb5e21e4f580bc15814964bca188e942.jpg

0: 640x640 (no detections), 137.7ms
Speed: 2.0ms preprocess, 137.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 135.8ms
Speed: 2.0ms preprocess, 135.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/datanew1.v2i.mul

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import pandas as pd
import numpy as np
import ast
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf

m = "rf"
m_version = '1'
my_model = joblib.load(f'new_{m}_model_{m_version}.pkl')
my_scaler = joblib.load(f'new_{m}_scaler_{m_version}.pkl')

dataset = pd.read_csv(f'C:/Users/User/Desktop/dataset/datanew1.v2i.multiclass/test_keypoints_0.7.csv')
df = dataset.drop(['image_name'], axis = 1)
df.head()
def convert_string_tuple_to_floats(s):
    if isinstance(s, str):
        return np.array(ast.literal_eval(s), dtype=np.float64)
    else:
        return s
# Apply the conversion to all relevant columns
for column in df.columns[:-1]:  # Assuming the last column is the target
    df[column] = df[column].apply(convert_string_tuple_to_floats)

X = np.array([np.hstack(row) for row in df.iloc[:,:-1].values])  # Flatten the arrays into a 2D array
y_test = df.iloc[:,-1].values
X = my_scaler.transform(X)
y_pred = my_model.predict(X)
# Assuming y_test and y_pred are already defined
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[317   1]
 [  4 219]]

Accuracy: 0.9907578558225508
Precision: 0.9954545454545455
Recall: 0.9820627802690582
F1 Score: 0.9887133182844242
